# MiniProyecto Parte 3
Ana Lucía Hernández - 17138
María Fernanda López - 17160

In [2]:
import numpy as np

In [3]:
class Server():
    def __init__(self, potencia, max_sol = 2400, cpus = 1):
        # potencia es la cantidad de solicitudes por segundo que puede tomar 
        # max_sol es la cantidad máxima de solicitudes que se recibirán * por minuto * (lambda_max)
        # cpus es la cantidad de servidores que tiene el sistema 
        self.potencia = potencia
        self.lambda_max = max_sol / 60 # se convierte solicitudes/min a solicitudes/seg
        self.cpus = cpus
    
    def next_ts(self, t): # los eventos serán solamente procesos de poisson no homogeneos, así que se programa solo para eso 
        while True:
            t = t - (np.log(np.random.uniform())/self.lambda_max)
            if np.random.uniform() <= np.random.poisson(t)/self.lambda_max:
                return t

    def simulate(self):
        # se asumirá que t0 = 0 y T = t0 + 60min = t0+ 3600seg
        t = 0 
        n = 0 # estado del sistema, numero de solicitudes en el sistema al tiempo t 
        T = 3600 # max tiempo de duracion es 1hr

        # contadores
        Na = 0 # llegadas 
        Nd = 0 # salidas

        A = [] # tiempos de llegada de la i-esima solicitud, ids son indices
        D = [] # tiempos de salida de la i-esima solicitud, ids son indices
        NT = [] # Tiempos de cada cliente en espera

        # eventos
        ta = self.next_ts(t) # tiempo de la proxima llegada
        td = np.zeros(self.cpus) + np.infty # set de tiempos de salida de cada servidor a infty, hay un td por cada server disponible
        busy_time = np.zeros(self.cpus) # tiempo que cada server estuvo ocupado
        served_by = [] # se guardan cuales solicitudes fueron atendidas por cuales server
        servers = np.zeros(self.cpus) # para llevar registro de quien esta ocupado y quien no

        while t < T: # el tiempo en el que estamos aún no excede el tiempo de cierre
            # CASO 1 
            # si el proximo tiempo de llegada es antes del proximo tiempo de salida, se encola
            if ta <= min(td):
                t = ta # nos posicionamos en el próximo tiempo de llegada
                Na += 1 # Contamos una llegada mas
                ta = self.next_ts(t) # calculamos el siguiente tiempo de llegada
                A.append(t) # guardamos el tiempo de la Na-esima llegada
                if n < self.cpus: # si hay menos clientes dentro que servidores, se le asigna uno que esté disponible
                    for i in range(self.cpus):
                        if servers[i] == 0: # Si el servidor esta disponible
                            NT = np.append(NT,t - A[len(A)-1])
                            # served_by.append(i) # se agrega el server que atendio el # de solicitud
                            td[i] = t + np.random.exponential(1/(self.potencia*60)) # calculamos su tiempo de salida y se lo asignamos a un servidor
                            busy_time[i] += td[i]-t # Calculamos el tiempo que va a estar en el servidor
                            servers[i] = 1 # seteamos el server como ocupado 
                            break;
                n += 1 # Contamos al nuevo cliente en el sistema
            
            # CASO 2
            # si el proximo tiempo de llegada es después del próximo tiempo de salida, se atiende ya que
            # se tiene la disponibilidad
            else:
                ti = np.argmin(td) # Escogemos al servidor que se desocupa primero
                t = td[ti] # avanzamos el tiempo al tiempo en que se desocupa
                served_by.append(ti) # se agrega el server que atendio el # de solicitud
                D.append(t)
                if n <= self.cpus: # Si hay menos o igual clientes que servidores
                    servers[ti] = 0 # liberamos el server
                    td[ti] = np.infty # seteamos el td a infty para indicar que aun no tiene otra solicitud
                else: # Si todavia hay mas clientes por atender
                    served_by.append(ti) # se agrega el server que atendio el # de solicitud
                    NT = np.append(NT,t - A[len(A)-1])
                    td[ti] = t + np.random.exponential(1/(self.potencia*60)) # Calculamos el tiempo de salida
                    busy_time[ti] += td[ti]-t # Calculamos el tiempo que va a estar en el servidor
                n -= 1 # Descontamos al cliente atendido del sistema
        # se calcula cuantas solicitudes atendio cada servidor 
        num_sol = np.zeros(self.cpus)
        for i in range(len(served_by)):
            num_sol[served_by[i]] += 1

        return { 
            "en_cola": NT, "num_sol": num_sol, "td": td, "A": A, "D": D, "busy_time": busy_time
        }

## Task 1 - Gorilla Megacomputing, 2400 max solicitudes

In [4]:
gorilla = Server(potencia = 100)
resultados = gorilla.simulate()

### ¿Cuántas solicitudes atendió cada servidor?





In [5]:
print("\tSolicitudes atendidas por servidor: ",resultados["num_sol"][0])

	Solicitudes atendidas por servidor:  143451.0


### ¿Cuánto tiempo estuvo cada servidor ocupado?

Ya que en este caso se tuvo un único servidor, el resultado también es único

In [6]:
print("\tTiempos de ocupacion de servidores: ", resultados["busy_time"][0])

	Tiempos de ocupacion de servidores:  23.7587915899135


### ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?

In [7]:
print("\tTiempo en el que el servidor estuvo libre: ",np.maximum(np.ones(gorilla.cpus)*60 - resultados["busy_time"],0)[0])

	Tiempo en el que el servidor estuvo libre:  36.2412084100865


### Cuánto tiempo en total estuvieron las solicitudes en cola?

In [8]:
print("\tTiempo total de solicitudes en cola: ",np.round(sum(resultados["en_cola"]),5))

	Tiempo total de solicitudes en cola:  0.16087


### En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [9]:
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(resultados["en_cola"]),5))

	Tiempo promedio de solicitudes en cola: 0.0


### En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

### ¿Cuál es el momento de la salida de la última solicitud?

In [10]:
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados["td"][-1],5), "seg")

	Tiempo de salida de la ultima solicitud atendida:  3600.00361 seg


## Task 3,4 - Gorilla Megacomputing, 6000 max solicitudes

In [11]:
gorilla = Server(potencia = 100, max_sol = 6000)
resultados = gorilla.simulate()

### ¿Cuántas solicitudes atendió cada servidor?





In [19]:
print("\tSolicitudes atendidas por servidor: ",resultados["num_sol"][0])

	Solicitudes atendidas por servidor:  360694.0


### ¿Cuánto tiempo estuvo cada servidor ocupado?

Ya que en este caso se tuvo un único servidor, el resultado también es único

In [13]:
print("\tTiempos de ocupacion de servidores: ", resultados["busy_time"][0])

	Tiempos de ocupacion de servidores:  59.12837801739507


### ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?

In [14]:
print("\tTiempo en el que el servidor estuvo libre: ",np.maximum(np.ones(gorilla.cpus)*60 - resultados["busy_time"],0)[0])

	Tiempo en el que el servidor estuvo libre:  0.8716219826049283


### Cuánto tiempo en total estuvieron las solicitudes en cola?

In [15]:
print("\tTiempo total de solicitudes en cola: ",np.round(sum(resultados["en_cola"]),5))

	Tiempo total de solicitudes en cola:  0.99286


### En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [16]:
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(resultados["en_cola"]),5))

	Tiempo promedio de solicitudes en cola: 0.0


### En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

### ¿Cuál es el momento de la salida de la última solicitud?

In [17]:
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados["td"][-1],5), "seg")

	Tiempo de salida de la ultima solicitud atendida:  3600.01182 seg
